In [8]:
from keras.models import model_from_json
from keras.preprocessing.image import array_to_img, img_to_array, load_img
import h5py
import os
from PIL import Image
from scipy.misc import imread, imresize, imsave
import numpy as np

print ('Mosaic: detect porngraghic')
print ('=====================================')

Mosaic: detect porngraghic


In [9]:
#import pic data
vPorn = '/mnt2/zhaihuixin/Frames/vPorn/'
vNonPornEasy = '/mnt2/zhaihuixin/Frames/vNonPornEasy/'
vNonPornDifficulty = '/mnt2/zhaihuixin/Frames/vNonPornDifficulty/'

#testPath = '/Users/holazhai/Desktop/'
def preprocess_image(image_path):
    try:
        img = imresize(imread(image_path), (64, 64))
    except:
        print 'error'
        return 'error'
    img = img[:, :, ::-1].astype('float64')
    img[:, :, 0] -= 103.939
    img[:, :, 1] -= 116.779
    img[:, :, 2] -= 123.68
    #print img.shape
    #img = img.transpose((2, 0, 1)).astype('float64')
    img = np.expand_dims(img[:,:,:3], axis=0)
    #print img.shape
    return img

In [10]:
def preprocess_data(path, labs):
    data = []
    label = []
    filelist = os.listdir(path)
    for f in filelist:
        preimage = preprocess_image(path + f)
        if preimage <> 'error':
            #print type(preimage)
            data.extend(preimage)
            label.extend([labs])
    return data, label

In [18]:
datavPorn, labelvPorn = preprocess_data(vPorn,1)
datavNonPornEasy, labelvNonPornEasy = preprocess_data(vNonPornEasy,0)
datavNonPornDifficulty, labelvNonPornDifficulty = preprocess_data(vNonPornDifficulty,0)

data = datavPorn
data.extend(datavNonPornEasy)
data.extend(datavNonPornDifficulty)

label = labelvPorn
label.extend(labelvNonPornEasy)
label.extend(labelvNonPornDifficulty)

data = np.array(data)

OSError: [Errno 2] No such file or directory: '/mnt2/zhaihuixin/Frames/vPorn/'

In [1]:
from keras.models import Sequential
from keras.layers import merge, Input
from keras.layers import Dense, Activation, Flatten, Dropout
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D, AveragePooling2D
from keras.layers import BatchNormalization

model = Sequential()
model.add(Convolution2D(64, 3, 3, input_shape=(64, 64, 3), name='convolution2d_1'))
model.add(Activation('relu', name='activation_1'))
model.add(MaxPooling2D((2, 2), strides=(2, 2), name='maxpooling2d_1'))

model.add(Convolution2D(64, 3, 3, name='convolution2d_2'))
model.add(Activation('relu', name='activation_2'))
model.add(MaxPooling2D((2, 2), strides=(2, 2), name='maxpooling2d_2'))

model.add(Flatten(name='flatten_1'))
model.add(Dense(128, name='dense_1'))
model.add(Activation('relu', name='activation_3'))
model.add(Dropout(0.5, name='dropout_1'))
model.add(Dense(128, name='dense_2'))
model.add(Activation('relu', name='activation_4'))
model.add(Dropout(0.5, name='dropout_2'))
model.add(Dense(1, name='dense_3'))
model.add(Activation('sigmoid', name='activation_5'))

model.compile(optimizer='Adadelta', loss='binary_crossentropy', metrics=['accuracy'])

Using TensorFlow backend.


In [100]:
model.fit(data, label, batch_size=128, nb_epoch=20,shuffle=True,verbose=1,show_accuracy=True,validation_split=0.11)
model.save_weights('/mnt2/zhaihuixin/src/jianhuang.h5')

Train on 1 samples, validate on 1 samples
Epoch 1/10
1/1 [==============================] - 0s - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 15.9424 - val_acc: 0.0000e+00
Epoch 2/10
1/1 [==============================] - 0s - loss: 16.1181 - acc: 0.0000e+00 - val_loss: 15.9424 - val_acc: 0.0000e+00
Epoch 3/10
1/1 [==============================] - 0s - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 15.9424 - val_acc: 0.0000e+00
Epoch 4/10
1/1 [==============================] - 0s - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 15.9424 - val_acc: 0.0000e+00
Epoch 5/10
1/1 [==============================] - 0s - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 15.9424 - val_acc: 0.0000e+00
Epoch 6/10
1/1 [==============================] - 0s - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 15.9424 - val_acc: 0.0000e+00
Epoch 7/10
1/1 [==============================] - 0s - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 15.9424 - val_acc: 0.0000e+00
Epoch 8/10
1/1 [==============================] - 0s - loss: 

In [22]:
print ('\r\nTry to Predict input images:')
print ('=====================================')

re_img = preprocess_image('/mnt2/zhaihuixin/6.jpg')  # this is a jpg image (size (1, 64, 64, 3))

print model.predict(re_img)
print model.predict_proba(re_img)

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_1 (Convolution2D)  (None, 62, 62, 64)    1792        convolution2d_input_1[0][0]      
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 62, 62, 64)    0           convolution2d_1[0][0]            
____________________________________________________________________________________________________
maxpooling2d_1 (MaxPooling2D)    (None, 31, 31, 64)    0           activation_1[0][0]               
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)  (None, 29, 29, 64)    36928       maxpooling2d_1[0][0]             
___________________________________________________________________________________________

In [ ]:
import os
import glob
path = '/mnt2/zhaihuixin/gol075/'
files = os.listdir(path) 
for f in files:
    x_img = preprocess_image(path+f)
    print (f + ' probability is: %3.3f%% ' %(100*model.predict_proba(x_img)))